# download your 9GAG collection

### step 1 initiate browser window

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import time 
import re
import pandas as pd
from bs4 import BeautifulSoup as b
import requests
import shutil
# DOWNLOAD AND PUT CHROMEDRIVER TO THIS NOTEBOOK FOLDER
driver = webdriver.Chrome(os.path.join('.', 'chromedriver'))
driver.get("https://9gag.com")

time.sleep(3)
driver.find_element_by_tag_name("body").send_keys("Keys.ESCAPE");


### step 2.1 manually login with facebook (haven't tested with other login options)

### step 2.2 proceed to your likes/upvotes page

![title](find_username.png)

In [3]:
USERNAME = 'your_username'
driver.find_element_by_tag_name("body").send_keys("Keys.ESCAPE");
driver.get(f"https://9gag.com/u/{USERNAME}/likes")

### step 3 run scroll-scrape process

In [4]:
# it's better to have your window active
print("it's better to have selenium window active")
def eat_streams(current_streams):
    global streams
    for s in current_streams:
        if not int(s.get('id').split('-')[1]) in streams:
            arts = []
            for a in b(str(s)).find_all('article'):
                art = {}
                art['post_id'] = str(a.get('id')).split('-')[2]

                ba = b(str(a))

                art['title'] = ba.find('h1').text

                head = ba.find('header').find('p', {'class': 'message'}).text
                art['section'] = re.sub('[\n ]+', ' ', head.split('·')[0]).strip()
                art['published'] = re.sub(' +', ' ', head.split('·')[1]).strip()

                meta = ba.find('p', {'class': 'post-meta'}).text
                art['upvotes'] = re.sub(' +', ' ', meta.split('·')[0]).strip()
                if art['upvotes'] != '• points':
                    art['upvotes'] = int(re.sub('(points|,|comments| )', '', art['upvotes']))
                else:
                    art['upvotes'] = 0 
                art['comments'] = re.sub(' +', ' ', meta.split('·')[1]).strip()
                art['comments'] = int(re.sub('(points|,|comments| )', '', art['comments']))

                video_src = ba.find('source', {'type':'video/mp4'})
                if video_src:
                    media_link = video_src.get('src')
                else:
                    img_src = ba.find('div', {'class':'post-container'})
                    img_src = b(str(img_src)).find('source', {'type':'image/webp'})
                    if img_src:
                        media_link = img_src.find_next('img').get('src')
                    else:
                        media_link = 'unable to find source tag. Html structure has probably changed since 2020/10/10'
                art['media_link'] = media_link
                arts.append(art)
            streams[int(s.get('id').split('-')[1])] = arts

_ = 0 
streams = {}
while 'No more posts' not in driver.page_source:
    _ += 1
    
    current_streams = b(driver.page_source).find_all('div', {'id': re.compile(r'^stream')})
    eat_streams(current_streams)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1.5)
    if not _ % 20: print(f"scrolled {_} times")
    # 300scrolls should be enough to load the whole page
    if _ > 300: break
print('scraping finished')

for stream_id, posts in streams.items():
    if stream_id == 0:
        df = pd.json_normalize(posts)
    else:
        df = df.append(pd.json_normalize(posts), ignore_index=True)

df['file_name']= df.index.astype('str').str.zfill(3) + '_ ' + df.title.str.replace('[:/\|?"*.%<>]','').apply(lambda x: x[:70])

fldr=f'9gags_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
if not os.path.exists(fldr):
    os.mkdir(fldr)

def download_media(columns):
    media_link = columns[0]
    file_name = columns[1]
    if media_link[-4:] in ('.jpg', '.mp4'):
        response = requests.get(media_link, stream=True)
        with open(os.path.join(fldr, file_name + media_link[-4:]), 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        return 1
    else:
        return 0

it's better to have selenium window active
scraping finished


### step 4 download the media-files, save csv

In [5]:
df['downloaded'] = df[['media_link','file_name']].apply(download_media, axis=1)
df.to_csv(os.path.join(fldr, '_db.csv'))
print('download finished')

download finished


### step 5 end. enjoy your lovely collection

## WARNING! CLEARING 9GAG PROFILE UPVOTES DATA
### step 6 (optional) clear your 9gag collection, post by post, to free up space for upcomming liked posts (remember, 9gag only saves 1000 upvoted posts in your profile)

### step 6.1 (optional) initiate browser window (skip if it's open)

In [7]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
# DOWNLOAD AND PUT CHROMEDRIVER TO THIS NOTEBOOK FOLDER
# ======= enable this block to prevent img loading:
# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.managed_default_content_settings.images": 2}
# chrome_options.add_experimental_option("prefs", prefs)
# driver = webdriver.Chrome(os.path.join('.', 'chromedriver'), options=chrome_options)
# :enable this block to prevent img loading =======
driver = webdriver.Chrome(os.path.join('.', 'chromedriver'))

driver.get("https://9gag.com")

time.sleep(3)
driver.find_element_by_tag_name("body").send_keys("Keys.ESCAPE");


### step 6.2 manually login with facebook (haven't tested with other login options) (skip if already logged in)

In [86]:
driver.find_element_by_tag_name("body").send_keys("Keys.ESCAPE");
USERNAME = 'your_username'
driver.get(f"https://9gag.com/u/{USERNAME}/likes")

### state-of-the-art code == contribution appreciated:
### step 6.3 click UNUPVOTE button

In [9]:
# it's better to have your window active
print("it's better to have selenium window active")

driver.get(f"https://9gag.com/u/{USERNAME}/likes")
from selenium.webdriver.common.keys import Keys

refresh_iteration = 0
while 'Posts you upvoted will show up here' not in driver.page_source:  
    driver.refresh()
    refresh_iteration += 1
    _ = 0 
    while 'No more posts' not in driver.page_source and 'Posts you upvoted will show up here' not in driver.page_source:
        _ += 1

        # unclick everything inside loaded streams
        # bye-bye to your upvoted gags
        up_active = driver.find_elements_by_xpath(f'''//a[@class="up active"]''')
        for element in up_active:
            driver.execute_script("arguments[0].click();", element)
            time.sleep(0.3)   


        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

        if not _ % 30: print(f"scrolled {_} times")
        # 300scrolls should be enough to load the whole page
        if _ > 300: break
    
    print(f'{refresh_iteration} page_refresh: since 9gag randomly throws [No more posts] block')
    time.sleep(0.6)   
    if refresh_iteration > 40: break
        
print('hopefully your collection is empty now. Otherwise, repeat untill you see the [Posts you upvoted will show up here] block')


it's better to have selenium window active
1 page_refresh: since 9gag randomly throws [No more posts] block
2 page_refresh: since 9gag randomly throws [No more posts] block
scrolled 30 times
3 page_refresh: since 9gag randomly throws [No more posts] block
scrolled 30 times
scrolled 60 times
scrolled 90 times
scrolled 120 times
scrolled 150 times
scrolled 180 times
scrolled 210 times
scrolled 240 times
scrolled 270 times
scrolled 300 times
4 page_refresh: since 9gag randomly throws [No more posts] block
hopefully your collection is empty now. Otherwise, repeat untill you see the [Posts you upvoted will show up here] block


### step 7 end. enjoy your empty 9GAG collection